In [45]:
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from joblib import dump
from joblib import load
import joblib


In [46]:
# データの読み込み

all_data = pd.read_csv('../data_processed/all_data1.csv')
submit_data = pd.read_csv('../data/submit_sample.csv')
ID = pd.read_csv('../data/test.csv')

test_data = all_data[27532:]
test_data = test_data.drop(['price'], axis=1)

In [47]:
path = 'weighted_ave(all_data1)'
# CatBoostのモデルをロード
catboost_model = CatBoostRegressor()
catboost_model.load_model(f'../model/{path}/catboost.cbm')

# LightGBMのモデルをロード
lightgbm_model = lgb.Booster(model_file=f'../model/{path}/lightGBM.bin')

# XGBoostのモデルをロード
xgboost_model = xgb.Booster()
xgboost_model.load_model(f'../model/{path}/XGBoost.xgb')


In [48]:
# 各モデルの予測を取得
cat_preds = catboost_model.predict(test_data)
lgb_preds = lightgbm_model.predict(test_data)
dval = xgb.DMatrix(test_data)
xgb_preds = xgboost_model.predict(dval)

# 重みのハイパーパラメータを設定
cat_weight = 0.2725346144167528
lgb_weight = 0.28264655483030904
xgb_weight = 0.4448188307529381

# 3つのモデルの予測を加重平均して最終的な予測を得る
test_predictions = (cat_preds * cat_weight + lgb_preds * lgb_weight + xgb_preds * xgb_weight) / (cat_weight + lgb_weight + xgb_weight)


In [49]:
# 提出用データフレームを作成
submission = pd.DataFrame({
    'id': ID['id'],
    'attendance': test_predictions
})

# ヘッダーを含まずにCSVファイルとして保存（'Deloitte Analytics/submission'フォルダに保存）
submission.to_csv(f'../submission/{path}.csv', index=False, header=False)